# Type Hints in Python
## 一、基本使用

In [1]:
a:int = 1
b:str = 'aaa'
c:list
c = [1, 2, 3]

def add(a:int, b:int)->int:
    return a + b

class Person:
    name:str
    sex:str = 'Male'
    age:int


### **注意**：只声明过类型提示的变量在没被赋值的情况下在运行时是不存在的

In [2]:
d: str
d

NameError: name 'd' is not defined

In [3]:
Person().name

AttributeError: 'Person' object has no attribute 'name'

## 二、如何获得类型提示信息
### 1. `__annotations__` 魔法方法

In [4]:
globals()['__annotations__']

{'a': int, 'b': str, 'c': list, 'd': str}

In [5]:
add.__annotations__

{'a': int, 'b': int, 'return': int}

In [6]:
Person.__annotations__

{'name': str, 'sex': str, 'age': int}

### 2.`inspect.signature()`方法

In [7]:
import inspect

In [8]:
inspect.signature(add)

<Signature (a: int, b: int) -> int>

### 3.`typing.get_type_hints()`方法

In [9]:
from typing import get_type_hints

In [10]:
get_type_hints(add)

{'a': int, 'b': int, 'return': int}

In [11]:
get_type_hints(Person)

{'name': str, 'sex': str, 'age': int}

## 三、可用作类型提示的类型
[概览](https://mypy.readthedocs.io/en/latest/cheat_sheet_py3.html)

1. 基础类型

In [12]:
a: int = 1
b: str = 'aaa'
c: list = [1, 2, 3]
d: dict = {'a': 'b'}
e: set = {1, 2, 3}
f: tuple = (1, 2, 3)

2. typing模块， 复杂类型

In [13]:
import typing

In [14]:
a: typing.List[int] = [1, 2, 3]
b: typing.Tuple[str, int, float] = ('aaa', 1, 2.5)
c: typing.Mapping[str, int] = {'aaa': 111, 'bbb': 2}

In [15]:
def func() -> typing.Callable[[int, int], int]:
    def add(a:int, b:int)-> int:
        return a + b

In [16]:
a:typing.Union[int, str] = 1

In [17]:
a:typing.Optional[str] = None

3. 使用字符串替代还未定义的类型

In [18]:
def say(a: 'Person') -> typing.NoReturn:
    a.say_hello()

class Person:
    def __init__(self, name):
        self.name = name
    def say_hello(self):
        print(f'{self.name} says:"Hello"')

typing.get_type_hints(say)

{'a': __main__.Person, 'return': typing.NoReturn}

4. 复杂的类型可以使用别名

In [19]:
PersonList = typing.List[Person]

def say_together(person_list: PersonList):
    for person in person_list:
        person.say_hello()

typing.get_type_hints(say_together)

{'person_list': typing.List[__main__.Person]}

## 四、类型提示的用处
### 1. 给人看
### 2. 静态分析 mypy
### 3. 编辑器和IDE的类型推导和错误提示
### 4. 一些实用场景

In [20]:
from abc import ABCMeta, abstractmethod

class Base(metaclass=ABCMeta):
    @abstractmethod
    def add(self, a:int, b:int)->int:
        ''''''
typing.get_type_hints(Base.add)

{'a': int, 'b': int, 'return': int}

In [21]:
from functools import singledispatch
@singledispatch
def add(a, b):
    ''''''
@add.register
def add_int(a:int, b:int):
    return a + b
add(1,2)

3

In [22]:
add('a', 'b')

In [23]:
@add.register
def add_str(a:str, b:str):
    return a + '_' + b
add(1,2)
add('a', 'b')

'a_b'

### 在Vibora框架中的用法
```python
from vibora import Vibora, Request
from vibora.responses import JsonResponse
​
app = Vibora()
​
@app.route('/')
async def home(request: Request):
    values = await request.json()
    print(values)
    return JsonResponse(values)
​
app.run()
```

## 五、如何检查对象的类型是否匹配类型提示
### 1. 一般类型和不带参数的typing模块的类型使用isinstance

In [24]:
isinstance(1, int)

True

In [25]:
isinstance([], typing.List)

True

### 2.对于带参数的typing模块的类型进行嵌套的检查

In [26]:
IntList = typing.List[int]

In [27]:
dir(IntList)

['__args__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__instancecheck__',
 '__le__',
 '__lt__',
 '__module__',
 '__mro_entries__',
 '__ne__',
 '__new__',
 '__origin__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasscheck__',
 '__subclasshook__',
 '__weakref__',
 '_inst',
 '_name',
 '_special',
 'copy_with']

In [28]:
IntList.__origin__

list

In [29]:
IntList.__args__

(int,)

In [30]:
StrIntTuple = typing.Tuple[str, int]

In [31]:
StrIntTuple.__origin__

tuple

In [32]:
StrIntTuple.__args__

(str, int)

In [33]:
OptionalInt = typing.Optional[int]

In [34]:
OptionalInt.__origin__

typing.Union

In [35]:
OptionalInt.__args__

(int, NoneType)

In [36]:
Text = typing.Union[str, bytes]

In [37]:
Text.__origin__

typing.Union

In [38]:
Text.__args__

(str, bytes)

#### 如何判断类型提示定义是否有参数

In [39]:
typing.List._special

True

In [40]:
typing.List[int]._special

False

In [41]:
typing.Optional._special

AttributeError: '_SpecialForm' object has no attribute '_special'

In [42]:
typing.Optional[int]._special

False